지난 시간, Day4_1_word2vec_and_doc2vec_moviereview.ipynb에서 영화 리뷰를 이용하여 172개의 영화의 임베딩 백터를 구하였습니다. 이 벡터를 이용하여 리뷰가 비슷한 영화들을 군집화 해보겠습니다. 

{영화 id: 영화 이름} dict와 pickling을 한 doc2vec model을 loading 하겠습니다. 이전 시간의 모델을 이용하기 때문에 이번 day5_1의 gensim version 역시 0.13.x입니다. version 1.0.1은 이후에 올려드리겠습니다. (이 튜토리얼을 만들고나서 또 gensim version이 2.x로 올랐습니다. 2.x로 만들어드리겠습니다)

id2movie의 형태는 아래와 같습니다. 

    {'98896': '교통신호',
     '75104': 'To',
     '22938': '제3공작',
     '28840': '허리케인 카터',
     '22757': '임 그리워',
     ...
    }

In [3]:
import pickle

with open('../../../data/sample_naver_movie/movie_review_doc2vec_model_v2.2.pkl', 'rb') as f:
    doc2vec_model = pickle.load(f)
    
with open('../../../data/sample_naver_movie/navermovie_info_idx2moviename.pkl', 'rb') as f:
    id2movie = pickle.load(f)

Doc2Vec에서 document (영화)의 임베딩 차원은 300이며, 영화의 개수는 172개임을 확인할 수 있습니다. 

In [4]:
doc2vec_model.docvecs.doctag_syn0.shape

(172, 100)

172개의 각 row에 해당하는 영화가 무엇인지 제목을 미리 list로 만들어 두겠습니다. 각 row id에 해당하는 label이 dict 형태로, Doc2Vec.docvecs.doctags에 저장되어 있습니다. 

    > doc2vec_model.docvecs.doctags
    
    {'MOVIE_100647': Doctag(offset=153, word_count=300107, doc_count=20577),
     'MOVIE_100691': Doctag(offset=155, word_count=310711, doc_count=19777),
     'MOVIE_100931': Doctag(offset=28, word_count=548150, doc_count=36068),
     'MOVIE_102272': Doctag(offset=84, word_count=580131, doc_count=33614),
     'MOVIE_102817': Doctag(offset=13, word_count=386768, doc_count=26021),
     'MOVIE_102824': Doctag(offset=103, word_count=227555, doc_count=13078),
     'MOVIE_102875': Doctag(offset=54, word_count=659849, doc_count=40894),
     ...
     }
     
위 예시에서 영화 id 100647의 row id는 offset=153, 153번 입니다. docvecs.doctags으로부터 아래의 코드를 실행하면 row 별 영화 이름을 얻을 수 있습니다. 
 
작업의 각 부분들의 내용을 확인하기 위해 step 별로 list의 3개의 값을 출력하였습니다. 
 

In [6]:
row2movie = sorted(doc2vec_model.docvecs.doctags.items(), key=lambda x:x[1].offset)
print('dict.items()를 하면 (key,value)의 list가 만들어집니다')
print(row2movie[:3], '\n')

row2movie = [row[0].split('_')[1] for row in row2movie]
print('row[0]에 의해 key들의 list가 만들어지고, str인 key를 _로 split한 뒤 [1]을 가져왔기 때문에 영화 id가 str 형식으로 만들어집니다')
print(row2movie[:3], '\n')

row2movie = [id2movie.get(row, None) for row in row2movie]
print('id2movie를 이용하여 str 형식의 영화 id를 영화 이름으로 바꿉니다')
print(row2movie[:3], '\n')

dict.items()를 하면 (key,value)의 list가 만들어집니다
[('MOVIE_72523', Doctag(offset=0, word_count=94513, doc_count=10187)), ('MOVIE_59845', Doctag(offset=1, word_count=144494, doc_count=13095)), ('MOVIE_109753', Doctag(offset=2, word_count=202367, doc_count=10361))] 

row[0]에 의해 key들의 list가 만들어지고, str인 key를 _로 split한 뒤 [1]을 가져왔기 때문에 영화 id가 str 형식으로 만들어집니다
['72523', '59845', '109753'] 

id2movie를 이용하여 str 형식의 영화 id를 영화 이름으로 바꿉니다
['고사 두 번째 이야기: 교생실습', '박쥐', '해무'] 



    KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm='auto')
    
k-means의 parameter는 위와 같습니다. 

- k의 값은 n_clusters입니다. 
- n_init은 반복 횟수이므로 1로 바꾸겠습니다. 
- 일반적으로 k-means는 엄청 큰 데이터라 하여도 10번, 혹은 수십번이면 거의 수렴합니다. 
- max_iter 역시 작은 숫자로 바꿔서 실행하시는게 좋습니다. 
- n_jobs는 multi-processing을 하기 위한 processor의 개수입니다. 수업 서버에서는 그대로 1을, 개인 작업 시에는 이용하고픈 processor의 개수를 입력합니다. 
- verbose = 1로 설정하면, 매 iteration마다 학습 상태가 출력됩니다. 속도를 측정할 때 용이합니다. 

또한, spherical k-means를 하기 위해서는 metrics='cosine'으로 바꿔야 하지만, k-means는 Euclidean을 기준으로 만들어졌습니다. 만약 spherical k-means를 하고 싶다면, (172, 300)의 행렬을 row normalize하여 unit vector로 만들기 바랍니다. 

우리는 172개의 영화를 10개의 그룹으로 만들어 보겠습니다. 172개를 10개로 나누는 것은 그리 어려운 작업은 아닌가봅니다. 그래서 max_iter=10이었지만, 실제 iteration은 그 전에 끝남을 볼 수 있습니다. 

In [7]:
from sklearn.preprocessing import normalize
movie_vectors = normalize(doc2vec_model.docvecs.doctag_syn0, axis=1, norm='l2')

In [8]:
%%time

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, max_iter=10, n_init=1, verbose=1)
print('kmeans.fit_predict()')
clusters = kmeans.fit_predict(movie_vectors)

print('\n\nkmeans.fit_transform()')
distance = kmeans.fit_transform(movie_vectors)

kmeans.fit_predict()
Initialization complete
Iteration  0, inertia 93.765
Iteration  1, inertia 59.825
Iteration  2, inertia 57.834
Iteration  3, inertia 56.870
Iteration  4, inertia 56.752
Iteration  5, inertia 56.704
Iteration  6, inertia 56.677
Converged at iteration 6


kmeans.fit_transform()
Initialization complete
Iteration  0, inertia 93.475
Iteration  1, inertia 59.968
Iteration  2, inertia 58.534
Iteration  3, inertia 58.140
Iteration  4, inertia 58.056
Converged at iteration 4
CPU times: user 132 ms, sys: 176 ms, total: 308 ms
Wall time: 123 ms


kmeans는 .fit_predict()와 .fit_transform() 두 가지 함수가 있습니다. 

    kmeans.fit_predict(doc2vec_model.docvecs.doctag_syn0)
    
fit_predict는 각 row의 label을 하나로 출력해줍니다. 

    kmeans.fit_transform(doc2vec_model.docvecs.doctag_syn0)
    
fit_transform은 각 row 별로 k 개의 centroid와의 거리를 출력해줍니다. 

In [9]:
clusters

array([6, 4, 3, 6, 9, 6, 7, 7, 8, 6, 6, 7, 9, 7, 1, 3, 4, 1, 9, 1, 5, 5, 2,
       2, 5, 2, 4, 3, 0, 5, 1, 9, 0, 2, 0, 7, 6, 5, 4, 2, 3, 4, 4, 2, 1, 7,
       5, 1, 8, 5, 8, 2, 9, 7, 2, 1, 7, 4, 1, 5, 6, 4, 6, 0, 7, 8, 1, 0, 0,
       5, 8, 7, 0, 5, 3, 9, 2, 1, 9, 1, 4, 7, 1, 6, 2, 0, 6, 7, 0, 6, 9, 6,
       2, 0, 1, 7, 1, 5, 1, 2, 1, 0, 1, 3, 0, 8, 6, 1, 5, 8, 4, 2, 3, 6, 0,
       1, 7, 5, 5, 6, 5, 6, 9, 8, 5, 9, 5, 1, 2, 4, 6, 0, 0, 5, 1, 7, 1, 5,
       8, 2, 0, 6, 7, 4, 0, 5, 7, 0, 4, 2, 7, 1, 5, 2, 3, 1, 9, 5, 9, 4, 8,
       8, 1, 6, 1, 1, 1, 5, 9, 2, 1, 9], dtype=int32)

In [10]:
distance.shape

(172, 10)

In [11]:
distance

array([[ 0.89482378,  1.00731586,  0.74531107, ...,  0.82845932,
         0.79268798,  1.02006966],
       [ 0.89232999,  0.84914317,  0.9146802 , ...,  0.94406952,
         0.74771188,  0.99522973],
       [ 0.74894126,  1.1274449 ,  1.13702065, ...,  0.95307312,
         0.61719052,  1.17676467],
       ..., 
       [ 0.71324029,  1.06601743,  1.18711702, ...,  1.10404612,
         0.86508808,  1.18010884],
       [ 1.08668984,  0.98177112,  1.04005369, ...,  0.77653645,
         1.04482512,  0.87507101],
       [ 1.17354924,  0.63829647,  0.69452872, ...,  0.71914574,
         1.03076123,  0.72523436]])

같은 cluster label을 지니는 영화별로 그룹을 묶어서 이름을 확인해봅니다. 그 뒤, 각 label 별로 몇 개의 영화가 묶여 있는지 개수를 출력합니다. 

In [12]:
from collections import defaultdict

cluster_to_row = defaultdict(lambda: [])
for row_id, label in enumerate(clusters):
    cluster_to_row[label].append(row_id)
    
cluster_to_row = dict(cluster_to_row)
for label, rows in cluster_to_row.items():
    print('cluster # %d has %d moveis' % (label, len(rows)))

cluster # 0 has 18 moveis
cluster # 1 has 29 moveis
cluster # 2 has 18 moveis
cluster # 3 has 8 moveis
cluster # 4 has 14 moveis
cluster # 5 has 23 moveis
cluster # 6 has 19 moveis
cluster # 7 has 18 moveis
cluster # 8 has 11 moveis
cluster # 9 has 14 moveis


각 cluster 별로 10개씩 영화의 이름을 출력하여, 군집화 결과가 어떤 느낌인지 확인해봅시다. 앞서 만들어둔 row2movie를 이용합니다. 

In [13]:
for label, rows in cluster_to_row.items():
    print('\ncluster # %d' % label)
    for row in rows[:10]:
        print('  > %s' % row2movie[row])


cluster # 0
  > 겨울왕국
  > 타이타닉
  > 지금, 만나러 갑니다
  > 클레멘타인
  > 인사이드 아웃
  > 세 얼간이
  > 비긴 어게인
  > 레미제라블
  > 어거스트 러쉬
  > 하울의 움직이는 성

cluster # 1
  > 배트맨 대 슈퍼맨: 저스티스의 시작
  > 메이즈 러너: 스코치 트라이얼
  > 캡틴 아메리카: 시빌 워
  > 빅 히어로
  > 인디펜던스 데이: 리써전스
  > 제이슨 본
  > 쥬라기 월드
  > 엑스맨: 데이즈 오브 퓨처 패스트
  > 워크래프트: 전쟁의 서막
  > 엣지 오브 투모로우

cluster # 2
  > 터널
  > 영웅: 샐러멘더의 비밀
  > 대호
  > 26년
  > 판도라
  > 천안함 프로젝트
  > 카트
  > 국제시장
  > 도가니
  > 연평해전

cluster # 3
  > 해무
  > 아가씨
  > 곡성(哭聲)
  > 검은 사제들
  > 더 테러 라이브
  > 숨바꼭질
  > 부산행
  > 설국열차

cluster # 4
  > 박쥐
  > 내가 살인범이다
  > 신세계
  > 의형제
  > 악마를 보았다
  > 좋은 놈, 나쁜 놈, 이상한 놈
  > 해바라기
  > 최종병기 활
  > 왕의 남자
  > 아저씨

cluster # 5
  > 과속스캔들
  > 미녀는 괴로워
  > 응답하라 1988
  > 완득이
  > 인턴
  > 광해, 왕이 된 남자
  > 님아, 그 강을 건너지 마오
  > 박수건달
  > 형
  > 두근두근 내 인생

cluster # 6
  > 고사 두 번째 이야기: 교생실습
  > 해운대
  > 국가대표
  > 미스터 고
  > 7광구
  > 포화 속으로
  > 감기
  > 고지전
  > 연가시
  > 초능력자

cluster # 7
  > 도둑들
  > 베를린
  > 럭키
  > 해적: 바다로 간 산적
  > 암살
  > 용의자
  > 검사외전
  > 관상
  > 감시자들
  > 역린

cluster # 8
  > 해리 포터와 죽음의 성물 - 

kmeans의 KMeans.cluster_centers_ 는 각 centroid의 좌표입니다. 

Doc2Vec 임베딩 벡터의 크기가 100이고, 클러스터의 개수가 10이기 때문에 centers 행렬의 크기는 (10, 100)입니다. 

In [14]:
kmeans.cluster_centers_.shape

(10, 100)

지난 시간에 연습했던 t-SNE를 이용하여 10개의 영화 군집을 시각화 할 수도 있습니다. label로 각 군집의 id와 영화 2개의 이름을 함께 annotation 합니다. 

영화의 개수가 172개 뿐이고, 300차원이기 때문에 10개의 군집으로 묶었을 때, 군집의 centroids의 벡터값이 서로 비슷할 수 있습니다. 이는 시각화의 측면에서는 좋지 않은 영향을 줄 수 있습니다. 시각화는 비슷한 것의 거리는 작게, 비슷하지 않은 것의 거리는 크게 표현되어야 하기 때문입니다. 하지만 k-means의 centroid는 고차원 벡터들의 평균이기 때문입니다. order 이상의 의미가 없을 수 있습니다. 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
krfont = {'family' : 'nanumgothic', 'weight' : 'bold', 'size'   : 10}
matplotlib.rc('font', **krfont)

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
y_tsne = tsne.fit_transform(kmeans.cluster_centers_)
y_tsne = y_tsne*1000


plt.figure(figsize=(15,10))
plt.scatter(y_tsne[:,0], y_tsne[:,1], color='yellow')
for i in range(y_tsne.shape[0]):
    one_sample = row2movie[cluster_to_row[i][0]]
    two_sample = row2movie[cluster_to_row[i][1]]
    plt.annotate('cluster %d \n(%s\n%s)' % (i, one_sample, two_sample), 
                 (y_tsne[i, 0], y_tsne[i, 1])
                )
plt.show()